In [156]:
import pandas as pd
import numpy as np
from statistics import mean
import time

##sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from emoji import UNICODE_EMOJI
import gensim.models as gsm


In [137]:
data = pd.read_csv("data/TrainV1.tsv",delimiter="\t", names = ['id','text','tag'])

In [138]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return indices_to_keep
idx_keep = clean_dataset(data)
data = data[idx_keep]

In [139]:
sample = [data.text, data.id]

In [140]:
sample

[0        pakistan ka ghra tauq pakistan israel ko tasle...
 1        madarchod mull ye mathura nahi dikha tha jab m...
 2        manya pradhan mantri mahoday shriman narendra ...
 3                  krishna jcb full trend chal rahi aa nan
 4        loksabha janta sirf modi ko vote de rahi thi n...
                                ...                        
 14589    what shame shame na kro yeh narendra modi thir...
 14590    hariom who bijli mantri peopl dont pay bill ka...
 14591                       bjp musalman ke naam pe kalank
 14592    mishra to phir bjp ke leader vika ke bare kyon...
 14593    iss ko wale doglepan i alawa kuch bhi nahi kareng
 Name: text, Length: 14570, dtype: object,
 0            3
 1           41
 2           48
 3           64
 4           66
          ...  
 14589    44673
 14590    45085
 14591    45096
 14592    45164
 14593    45220
 Name: id, Length: 14570, dtype: int64]

In [141]:
tweets = sample[:100]

In [142]:
tweets_dict = dict(zip(tweets[1], tweets[0])) 

In [143]:
tweets_dict

{3: 'pakistan ka ghra tauq pakistan israel ko tasleem nahein kerta isko palestin kehta - occupi palestin nan',
 41: 'madarchod mull ye mathura nahi dikha tha jab mullo ne hindu ko iss liy mara ki vo lass ki pais mag liy',
 48: 'manya pradhan mantri mahoday shriman narendra modi ji pradhanmantri bann par hardik badhai tahe dil',
 64: 'krishna jcb full trend chal rahi aa nan',
 66: 'loksabha janta sirf modi ko vote de rahi thi na ki kisi mp bjp ko without',
 68: 'bhosdik tum pechvad ki tatti hi rahog bc nan',
 90: 'love u bhaijan ... ♥♥ father + son bharat iambharat bharatthiseid best pic from entir promot mashallah',
 99: 'tumhara pass abh deemagh hai nahi islea googl ko apna deemagh banaya hua hai har koi tumhari tarh',
 101: 'nolo weni anker gae weekend 😂😂😂😂😂 nan',
 104: 'aimim lage raho mullo tumhr issi quran faad gyan ki kayal hain duniya allah bhi khus',
 106: 'omer muhammadpti pakhtoon k haqooq ki jang nahi punjab k haqooq ki jung pak',
 146: 'ye sale desh ke kabhi sage nahin ho s

## Feature 1 - Emojis

In [144]:
feature_emo = [[0]*300]*len(tweets[0])

In [145]:
def has_emoji(s):
    emo = emo_rgx.findall(s)
    return emo

In [146]:
emo_rgx = emoji.get_emoji_regexp()

### emoji2vec

In [147]:
## ## from https://github.com/uclnlp/emoji2vec
e2v = gsm.KeyedVectors.load_word2vec_format('lib/emoji2vec.bin', binary=True)
e2v_vocab = e2v.vocab

In [148]:
# for tweet in tweets_dict:
#     print(tweets_dict[tweet])

In [149]:
data

id                                               text  tag
0          3  pakistan ka ghra tauq pakistan israel ko tasle...    0
1         41  madarchod mull ye mathura nahi dikha tha jab m...    0
2         48  manya pradhan mantri mahoday shriman narendra ...    2
3         64            krishna jcb full trend chal rahi aa nan    2
4         66  loksabha janta sirf modi ko vote de rahi thi n...    2
...      ...                                                ...  ...
14589  44673  what shame shame na kro yeh narendra modi thir...    0
14590  45085  hariom who bijli mantri peopl dont pay bill ka...    0
14591  45096                     bjp musalman ke naam pe kalank    0
14592  45164  mishra to phir bjp ke leader vika ke bare kyon...    0
14593  45220  iss ko wale doglepan i alawa kuch bhi nahi kareng    0

[14570 rows x 3 columns]

## Word Embeddings

In [40]:
# Create feature vectors
vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)

vectors = vectorizer.fit_transform(data['text'])

In [152]:
final_vector = feature_emo

## Classifier

In [153]:
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(final_vector, data.tag, test_size=0.3,random_state=109) # 70% training and 30% test

In [ ]:
classifier_linear = svm.SVC(kernel='linear')
scores = mean(cross_val_score(classifier_linear, X_train, y_train, cv = 5))
print("F1",scores)

In [133]:
# # Perform classification with SVM, kernel=linear
# classifier_linear = svm.SVC(kernel='linear')
# t0 = time.time()
# classifier_linear.fit(X_train, y_train)
# t1 = time.time()
# prediction_linear = classifier_linear.predict(X_test)
# t2 = time.time()
# time_linear_train = t1-t0
# time_linear_predict = t2-t1

In [ ]:
# # results
# print("Results for SVC(kernel=linear)")
# print("Training time: %fs; Prediction time: %fs" % (time_linear_train, time_linear_predict))
# report = classification_report(y_test, prediction_linear, output_dict=True)
# report